## Quantifying atmospheric jitter with trailed images

Craig Lage - 25-Apr-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.bestEffort import BestEffortIsr
from lsst.summit.utils.plotting import plot
from lsst.summit.utils import quickSmooth
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from scipy.optimize import minimize

In [ ]:
butler = butlerUtils.makeDefaultLatissButler(embargo=True)

In [ ]:
dataId = dict(day_obs=20240306, detector=0, seq_num=566)
bestEffort = BestEffortIsr(embargo=True)
bestEffort.getExposure(dataId)
dataType = 'quickLookExp'
exp = butler.get(dataType, dataId)

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(10,10))
plot(exp, figure=fig, stretch='ccs')
plt.savefig("/home/c/cslage/u/AuxTel/jitter/Jitter_from_Stopped_Drive_20240306_566.png")

In [ ]:
xmin = 2500; xmax = 2750
ys = [700, 1200, 1900, 2200]
fig, axs = plt.subplots(2,2, figsize = (10,10))
plt.subplots_adjust(wspace=0.3, hspace=0.3)
plt.suptitle("Fits to Trailing streak. AuxTel 20240306, seqNum 566", fontsize=18)
for i, y in enumerate(ys):
    axY = i%2
    axX = int(i/2)
    data = arr[y, xmin:xmax]
    axs[axX][axY].plot(data, label="Pixel data")
    guess = np.polyval(fit, y)
    param0 = [5.0, 10.0, guess, 5.0]
    bounds = [[2.0, 10.0], [1.0, 1000.0], [guess-10, guess+10], [0.1, 20.0]]
    args = [data]
    bestFit = minimize(fitFunc, param0, args=args, bounds=bounds, method='Powell')
    [const, mag, peak, sigma] = bestFit.x
    xplot = np.linspace(0, len(data), 500)
    yplot = const + mag * np.exp(-(xplot - peak)**2/(2 * sigma)**2)
    axs[axX][axY].plot(xplot, yplot, color='red', ls = '--', label='Fit')
    axs[axX][axY].set_title(f"Y = {y}, Peak = {peak:.2f}")
    axs[axX][axY].set_ylabel("Flux(ADU)")
    axs[axX][axY].set_xlabel(f"Xpixel - {xmin}")
    axs[axX][axY].legend()
    #print(y, const, mag, peak, sigma, flush=True)
plt.savefig("/home/c/cslage/u/AuxTel/jitter/Streak_Fits_20240306_566.png")

In [ ]:
def fitFunc(params, args):
    [const, mag, peak, sigma] = params
    [data] = args
    diff = 0.0
    for i, value in enumerate(data):
        model = const + mag * np.exp(-(i - peak)**2/(2 * sigma)**2)
        diff += (model - value)**2
    return diff

arr = exp.image.array
peaks = []
ys = []
intercept = 0.070; slope = 18.713 # this is the fit to the XY line, obtained by iterating
fit = np.array([intercept, slope])
for y in range(700, 2400):
    guess_peak = np.polyval(fit, y)
    param0 = [5.0, 10.0, guess_peak, 5.0]
    bounds = [[2.0, 10.0], [1.0, 1000.0], [guess_peak - 10, guess_peak + 10], [0.1, 20.0]]
    data = arr[y, 2500:2750]
    args = [data]
    bestFit = minimize(fitFunc, param0, args=args, bounds=bounds, method='Powell')
    [const, mag, peak, sigma] = bestFit.x
    #print(y, peak, flush=True)
    ys.append(y)
    peaks.append(peak)
        
        
    
    

In [ ]:
fig = plt.figure(figsize=(10,5))
plt.suptitle("Jitter of trailing streak AuxTel 20240306, seqNum 566")
plt.subplot(1,2,1)
plt.title("Streak Jitter")
plt.plot(ys, peaks)
fit = np.polyfit(ys, peaks, deg=1)
xplot = range(700,2400)
yplot = np.polyval(fit, xplot)
plt.plot(xplot, yplot, color='red', ls='--')
diffs = []
for i in range(len(xplot)):
    diffs.append(peaks[i] - yplot[i])
plt.xlabel("Y (pixels)")
plt.ylabel(f"Streak X peak (pixels - {xmin})")
plt.text(750, 160, f"Streak sigma = {(np.std(diffs) * 0.1):.2f} arcseconds")
plt.subplot(1,2,2)
plt.title("Deviation of peak from linear fit")
plt.hist(diffs, bins=50)
plt.xlabel("Pixels")
plt.savefig("/home/c/cslage/u/AuxTel/jitter/Streak_Jitter_20240306_566.png")